In [ ]:
from Series import Speed, Direction
from Time import Hours, Date, DateTime
from DataFrame import PriceDemand, Cities

import pandas as pd
from datetime import date, datetime, time, timedelta
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

price surge for one day
price surge for 9am and 3pm seperately 

price_surge_score_3pm = for all price surges in that day, take sum(1 / | time of surge - 3pm | )


fake_value_lookup = {"Evaporation" : [row_index1, row_index2, ... ]}

testing correlations between features


test correlation of features with a label 


1) process features, normalise, fill missing etc..

2) test different approaches for choosing a label





In [11]:
cities = Cities()
price = PriceDemand()
cities.join_price(price.get_grouped())

,Date,REGION,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,...,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa),TOTALDEMAND,PRICESURGE
1695,2021-02-01,NSW1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7849.959149,0
1695,2021-02-02,NSW1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7399.210208,0
1695,2021-02-03,NSW1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7235.404583,0
1695,2021-02-04,NSW1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7824.979583,0
1695,2021-02-05,NSW1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8197.782708,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,2022-03-14,VIC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4370.207222,0
1695,2022-03-15,VIC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5005.891840,0
1695,2022-03-16,VIC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4832.505903,0
1695,2022-03-17,VIC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5101.258750,0


In [12]:
cities.mean_fill_nans(cities.columns)

In [ ]:
cities_df = cities.df